## Leaky integrated firing neuron model  
## Few feautures :
### Spatial network, no memory term but exponential decay of cumulative dose

This is a minor change of the Dodds and Watts model  
Implement adaptive rewiring and generate statistics

In [19]:
#importing the required libraries

import networkx as nx
import matplotlib.pyplot as plt #for plotting
import numpy as np #for using arrays and vectorising the code wherever possible
import scipy
import random
# from numba import jit #numba precomplier to make the code faster
import pandas as pd
import copy

# Variables used in the simulation

In [20]:
N = 100
gamma = 5 #decay of cumulative dose per unit time (second)
T = 100 #total number of seconds
dt = 0.1 #seconds
'''number of simulation steps is T/dt'''

'''Dodds watts parameters'''
# p = 10 #rate of dose transfer (per unit time (second))
# p = p*dt #rate of dose transfer per unit simulation step
r = 10
r = r*dt
rho = 10
rho = rho*dt

d_star = 1
dose_quantity = 1
D_ini = 5

'''network parameters'''
circ_rad = 100 #for a network in which each node is connected to other nodes withing a circle of radius circ_rad
nn = 7 #the number of nearest neighbours(i.e. k) in knn
average_degree = 6

'''rewiring parameters'''
# p_rew = 5 #rate of rewiring(per unit time(second))
# p_rew = p_rew*dt

'rewiring parameters'

# Creating the network. 
### The network outputs coordinates, distance matrix, edge_list and adjacency matrix

In [21]:
'''
obtains the number of nodes N
generates N points
calculates euclidean distance between each pair of points
returns the coordinates of the points and the distance matrix which is N*N
'''
def calc_dist(N) :
    coords = []
    for counter in range(N) :
        coords.append((N*np.random.random(),N*np.random.random()))

    dist_mat = scipy.spatial.distance.cdist(coords,coords)
    return dist_mat,coords

### 1. metric network

In [22]:
'''network = connect to points withing a circle of radius'''
#creates a link between points/nodes which satisfies the conditions of the network
def coupling(dist_mat,circ_rad) :
    edge_list = []
    edge_mask = (dist_mat<circ_rad) & (dist_mat>0)
    edge_list.extend(np.ndarray.tolist(np.transpose(np.where(edge_mask))))
#     print(type(np.ndarray.tolist(np.transpose(np.where(edge_mask)))))
    return edge_list

### 2. K nearest neighbour network

In [23]:
'''network = knn'''
def knn(dist_mat,nn,N) :
    near_neigh = np.argsort(dist_mat)
    selec_near_neigh = np.zeros((N,nn))
    selec_near_neigh = near_neigh[:,0:nn+1]

    edge_list = []
    for i in range(N) :
        for j in range(1,nn+1) :
            link = [i,selec_near_neigh[i,j]]
            edge_list.append(link)

    return edge_list

### 3. Random Network

In [ ]:
'''random network'''
def rand_network(average_degree,N) :
    z1 = np.random.uniform(size = (N,N))
    E,F = np.meshgrid(np.arange(0,N),np.arange(0,N))
    mask = ((average_degree/N) > z1) & (E!=F)
    adjacency_matrix = np.int64(np.zeros(shape=(N,N)))
    adjacency_matrix[mask] = np.int64(1)
    edge_list = []
    edge_list.extend(np.ndarray.tolist(np.transpose(np.where(adjacency_matrix==1))))
    return edge_list

# Function defintions required for the main part of the script

In [ ]:
# # @jit(nopython=True)
# '''infecting the left part of space.'''
# def left_part_infec(N) :
#     x_coord = []
#     y_coord = []
#     for j in range(len(coords)) :
#         x_coord.append(coords[j][0])
#         y_coord.append(coords[j][1])
#     x_coord = np.asarray(x_coord)
#     y_coord = np.asarray(y_coord)
#     points = np.asarray(np.where(x_coord<(N/5))) 
#     indi_state[points.T] = 2

#     return indi_state

In [ ]:
'''infecting connected nodes'''
def InfectNetworkNeighbors(net,seed_node,init_infected_nodes):
     # if in bulk find one node randomly, and infect its neighbours
    infected_nodes = set()
    candidate_nodes = set()
    explored_nodes = set()

    #pick the seed node
    infected_nodes.add(seed_node)
    explored_nodes.add(seed_node)

    curr_node=seed_node

    #add its neighbors to the list of candidates
    for n in net.neighbors(curr_node):
        candidate_nodes.add(int(n))
    #print( curr_node)
    #print( candidate_nodes)

    #while we need to select more nodes...
    while len(infected_nodes) < init_infected_nodes:

        #if there are candidate nodes, select one of them
        if(len(candidate_nodes) > 0):
            new_node = np.random.choice(list(candidate_nodes),1)[0]
            infected_nodes.add(new_node)
            candidate_nodes.remove(new_node)

        elif len(infected_nodes - explored_nodes) > 0:
            curr_node = np.random.choice(list(infected_nodes -
            explored_nodes),1)[0]
            explored_nodes.add(curr_node)
            for n in set(net.neighbors(curr_node)) - infected_nodes:
                candidate_nodes.add(n)

        else:
            print('Initial node infection step failed')
            return None
    return infected_nodes

In [ ]:
''' function to transfer doses'''
# @jit(nopython=True)
def dose(adj_mat,p,partner_state_prev,d) :

    z1 = np.random.uniform(size=(len(adj_mat),len(adj_mat[0])))
    
    dose_transfer_mask = p > z1
    mod_adj_mat = np.multiply(adj_mat,z1) #modifying adjacency matrix to include the proability of dose transfer
    dose_mask = (p>mod_adj_mat) & (adj_mat!=0) & (partner_state_prev == 2)
    d[dose_mask] = dose_quantity #whenever it is proabable, dose transfer occurs
    
    return d

In [ ]:
'''rewiring function which returns a new adjacency matrix'''

def rew(p_rew,adjacency_matrix,indi_state_for_rew_prev) :
    
    z1_rew = np.random.uniform(size=(len(adj_mat),len(adj_mat[0])))
    
    rew_mask_plus = (p_rew > z1_rew)  & (indi_state_for_rew_prev == 1)
    #create links. Dont break any.
    adjacency_matrix[rew_mask_plus] = 1
    rew_mask_minus = (p_rew > z1_rew) & (indi_state_for_rew_prev == 2)
    #break links. Dont create any.
    adjacency_matrix[rew_mask_minus] = 0
    
    np.fill_diagonal(adjacency_matrix,0)
    return adjacency_matrix

In [ ]:
'''function to update cumulative doses'''
def cumu_dose(d,D_prev,gamma) :
    I = d.sum(axis=1).reshape(N,1)
#     I = 0
    D = (D_prev - (gamma*D_prev*dt)) + I
    return D

In [ ]:
'''function to update the states of the nodes of the system'''
def upd_indi_state(D,d_star,indi_state_prev) :
    
    z2 = np.random.uniform(size=(N,1))
    z3 = np.random.uniform(size=(N,1))
    
    indi_state = indi_state_prev
    
    indi_state_mask1 = (D>=d_star) & (indi_state_prev==1)
    indi_state[indi_state_mask1] = 2
    
    indi_state_mask21 = (D<d_star) & (indi_state_prev==2) & (r>=z2) & (rho>=z3)
    indi_state[indi_state_mask21] = 1
    
    indi_state_mask22 = (D<d_star) & (indi_state_prev==2) & (r>=z2) & (rho<z3)
    indi_state[indi_state_mask22] = 3
    
    indi_state_mask23 = (D<d_star) & (indi_state_prev==2) & (r<z2)
    indi_state[indi_state_mask23] = 2
    
    return indi_state

In [ ]:
'''function to get/identify the infected nodes and the suceptible nodes'''
def states(indi_state) :
    infec_indi = []
    suscep_indi = []
    infec_indi = np.transpose(np.where(indi_state==2))
    suscep_indi = np.transpose(np.where(indi_state==1))
    return infec_indi,suscep_indi

In [ ]:
def networkx_graph(coords,edge_list) :
    G = nx.DiGraph()
    pos = {(i): (coords[i][0],coords[i][1]) for i in range(N)}
    G.add_nodes_from(pos.keys())
    G.add_edges_from(edge_list)
    return G

In [ ]:
def edge_from_adj_mat(adj_mat_list,q):
    edge_list = []
    edge_list.extend(np.ndarray.tolist(np.transpose(np.where(adj_mat_list[q]==1))))
    return edge_list

# Choose which network you want in the program
### Coupling network of knn network?

In [ ]:
'''returns coordinates, distance matrix, edge_list and the adjacency matrix'''

dist_mat,coords = calc_dist(N)
# edge_list = coupling(dist_mat,circ_rad)
# edge_list = knn(dist_mat,nn,N)
edge_list = rand_network(average_degree,N)

org_adj_mat = np.int64(np.zeros((N,N)))
for i in range(len(edge_list)):
    org_adj_mat[edge_list[i][0],edge_list[i][1]] = np.int64(1)

# The main part of the script

In [ ]:
#infected state time series data frame
#rows indicate parameter (p,start)
#columns indicate
timeseries_infec_frac = pd.DataFrame() 

In [ ]:
df_row = 0
for p_rew in range(0,11) :
    p_rew = p_rew*dt        
    for p in range(0,11) : #rate of dose transfer (per unit time (second))
        p = p*dt #rate of dose transfer per unit simulation step
        for start in np.arange(1,11) : #number of nodes to infect initially
            print('(p_rew,p,start) = ', (p_rew,p,start))
            q = 0 #to generate new edge_list from the new adjacency matrix 

            t = np.arange(0,T,dt)

            numpy_array = np.zeros((N,len(t)),dtype = np.int64) #stores the states of the nodes. Rows are nodes and columns are time steps
            D_array = np.zeros((N,len(t))) #stores the cumulative doses. Rows are nodes and columns are time steps

            adj_mat_list = []
            adj_mat = org_adj_mat
            adj_mat_new = copy.deepcopy(adj_mat)
            adj_mat_list.append(adj_mat_new) #list of arrays which shows the time series of the adjacency matrix

            #coords remain the same. Get new edge_list from the latest adjacency matrix
            edge_list = edge_from_adj_mat(adj_mat_list,q)
            G = networkx_graph(coords,edge_list) #networkx graph

            indi_state = np.random.randint(1,2,size=(N,1))
            infected_nodes = None
            while infected_nodes is None : #infecting 'start' number of network neighbours
                infected_nodes = InfectNetworkNeighbors(G,np.random.randint(N),start)
                print(infected_nodes)
            infected_nodes = np.asarray(list(infected_nodes)).reshape(len(infected_nodes),1)
            indi_state[infected_nodes[:,0],0] = 2

            A,B = np.meshgrid(indi_state,indi_state) 
            partner_state = A
            indi_state_for_rew = B

            numpy_array [:,0] = indi_state[:,0]

            indi_state_prev = indi_state
            partner_state_prev = partner_state
            indi_state_for_rew_prev = indi_state_for_rew

            d = np.zeros((len(adj_mat),len(adj_mat[0])))

            D = np.zeros((N,1))
            D[np.where(indi_state==2)] = D_ini
            D_array[:,0] = D[:,0]
            D_prev = D

            infec_frac = np.count_nonzero(indi_state == 2)/N
            timeseries_infec_frac.loc[df_row,0] = infec_frac
            '''the main part of the simulation'''
            counter = 0
            df_col = 1
            for t in np.arange(dt,T,dt) :
                q = q + 1 
                counter = counter + 1
                infec_indi = []
                suscep_indi = []
                d = np.zeros((len(adj_mat),len(adj_mat[0])))
                d = dose(adj_mat,p,partner_state_prev,d)

                adj_mat = rew(p_rew,adj_mat,indi_state_for_rew_prev)
                adj_mat_new = copy.deepcopy(adj_mat)
                adj_mat_list.append(adj_mat_new)

                #coords remain the same. Get new edge_list from the latest adjacency matrix
                edge_list = edge_from_adj_mat(adj_mat_list,q)
                G = networkx_graph(coords,edge_list) #networkx graph

                D = cumu_dose(d,D_prev,gamma)
                D_array[:,counter] = D[:,0]

                indi_state = upd_indi_state(D,d_star,indi_state_prev)
                infec_frac = np.count_nonzero(indi_state == 2)/N
                timeseries_infec_frac.loc[df_row,df_col] = infec_frac

                numpy_array[:,counter] = indi_state[:,0]
                A,B = np.meshgrid(indi_state,indi_state)

                infec_indi, suscep_indi = states(indi_state)

                partner_state = A
                indi_state_for_rew = B
                indi_state_prev = indi_state
                partner_state_prev = partner_state
                indi_state_for_rew_prev = indi_state_for_rew

                D_prev = D
                D = np.zeros((N,1))
                df_col = df_col + 1 

            df_row = df_row + 1 #going to the next row of the df to store timeseries of next (p,start)

    filename = 'file_avdeg_'+str(average_degree)+'_ini_'+str(np.int64(0))+'_'+str(np.int64(11))+'.h5'
    key_val = 'p_rew_'+str(np.int64(p_rew/dt))
    timeseries_infec_frac.to_hdf(filename, key = key_val, mode='a')
    df_row = 0

(p_rew,p,start) =  (0.0, 0.0, 1)
{72}
(p_rew,p,start) =  (0.0, 0.0, 2)
{32, 63}
(p_rew,p,start) =  (0.0, 0.0, 3)
{48, 68, 54}
(p_rew,p,start) =  (0.0, 0.0, 4)
{17, 66, 68, 61}
(p_rew,p,start) =  (0.0, 0.0, 5)
{47, 84, 53, 90, 61}
(p_rew,p,start) =  (0.0, 0.0, 6)
{32, 98, 17, 55, 27, 29}
(p_rew,p,start) =  (0.0, 0.0, 7)
{64, 36, 46, 18, 88, 90, 59}
(p_rew,p,start) =  (0.0, 0.0, 8)
{97, 8, 77, 14, 45, 84, 91, 92}
(p_rew,p,start) =  (0.0, 0.0, 9)
{67, 36, 69, 6, 40, 9, 19, 25, 60}
(p_rew,p,start) =  (0.0, 0.0, 10)
{32, 68, 39, 81, 82, 20, 53, 55, 57, 25}
(p_rew,p,start) =  (0.0, 0.1, 1)
{80}
(p_rew,p,start) =  (0.0, 0.1, 2)
{88, 6}
(p_rew,p,start) =  (0.0, 0.1, 3)
{25, 2, 28}
(p_rew,p,start) =  (0.0, 0.1, 4)
{27, 45, 22, 31}
(p_rew,p,start) =  (0.0, 0.1, 5)
{32, 35, 71, 15, 93}
(p_rew,p,start) =  (0.0, 0.1, 6)
{42, 75, 12, 53, 22, 27}
(p_rew,p,start) =  (0.0, 0.1, 7)
{32, 98, 17, 55, 27, 29, 31}
(p_rew,p,start) =  (0.0, 0.1, 8)
{0, 66, 38, 8, 41, 80, 56, 92}
(p_rew,p,start) =  (0.0, 0.1, 

(p_rew,p,start) =  (0.1, 0.30000000000000004, 1)
{51}
(p_rew,p,start) =  (0.1, 0.30000000000000004, 2)
{19, 55}
(p_rew,p,start) =  (0.1, 0.30000000000000004, 3)
{17, 59, 92}
(p_rew,p,start) =  (0.1, 0.30000000000000004, 4)
{96, 97, 98, 55}
(p_rew,p,start) =  (0.1, 0.30000000000000004, 5)
{33, 99, 44, 23, 90}
(p_rew,p,start) =  (0.1, 0.30000000000000004, 6)
{33, 74, 12, 89, 58, 27}
(p_rew,p,start) =  (0.1, 0.30000000000000004, 7)
{35, 69, 6, 76, 52, 86, 55}
(p_rew,p,start) =  (0.1, 0.30000000000000004, 8)
{98, 41, 76, 48, 87, 56, 92, 63}
(p_rew,p,start) =  (0.1, 0.30000000000000004, 9)
{5, 39, 72, 7, 71, 77, 16, 50, 19}
(p_rew,p,start) =  (0.1, 0.30000000000000004, 10)
{66, 67, 36, 98, 38, 12, 14, 17, 84, 92}
(p_rew,p,start) =  (0.1, 0.4, 1)
{48}
(p_rew,p,start) =  (0.1, 0.4, 2)
{41, 75}
(p_rew,p,start) =  (0.1, 0.4, 3)
{81, 99, 86}
(p_rew,p,start) =  (0.1, 0.4, 4)
{2, 5, 71, 31}
(p_rew,p,start) =  (0.1, 0.4, 5)
{64, 9, 78, 18, 22}
(p_rew,p,start) =  (0.1, 0.4, 6)
{70, 79, 48, 88, 89, 9

(p_rew,p,start) =  (0.2, 0.5, 6)
{37, 44, 45, 48, 25, 31}
(p_rew,p,start) =  (0.2, 0.5, 7)
{2, 3, 39, 20, 88, 25, 63}
(p_rew,p,start) =  (0.2, 0.5, 8)
{67, 37, 39, 9, 73, 81, 50, 30}
(p_rew,p,start) =  (0.2, 0.5, 9)
{34, 35, 74, 47, 30, 57, 63, 95, 31}
(p_rew,p,start) =  (0.2, 0.5, 10)
Initial node infection step failed
None
{64, 96, 36, 5, 6, 71, 42, 19, 58, 27}
(p_rew,p,start) =  (0.2, 0.6000000000000001, 1)
{1}
(p_rew,p,start) =  (0.2, 0.6000000000000001, 2)
{49, 60}
(p_rew,p,start) =  (0.2, 0.6000000000000001, 3)
{52, 55, 79}
(p_rew,p,start) =  (0.2, 0.6000000000000001, 4)
{89, 60, 62, 25}
(p_rew,p,start) =  (0.2, 0.6000000000000001, 5)
{70, 10, 75, 52, 22}
(p_rew,p,start) =  (0.2, 0.6000000000000001, 6)
{66, 9, 49, 83, 61, 31}
(p_rew,p,start) =  (0.2, 0.6000000000000001, 7)
{64, 1, 96, 74, 14, 85, 27}
(p_rew,p,start) =  (0.2, 0.6000000000000001, 8)
Initial node infection step failed
None
{66, 11, 79, 81, 18, 83, 21, 88}
(p_rew,p,start) =  (0.2, 0.6000000000000001, 9)
{32, 97, 65, 

(p_rew,p,start) =  (0.30000000000000004, 0.5, 8)
{96, 4, 27, 75, 86, 56, 88, 95}
(p_rew,p,start) =  (0.30000000000000004, 0.5, 9)
{67, 70, 8, 42, 76, 14, 48, 83, 86}
(p_rew,p,start) =  (0.30000000000000004, 0.5, 10)
{64, 34, 35, 73, 13, 80, 48, 87, 56, 63}
(p_rew,p,start) =  (0.30000000000000004, 0.6000000000000001, 1)
{95}
(p_rew,p,start) =  (0.30000000000000004, 0.6000000000000001, 2)
{0, 15}
(p_rew,p,start) =  (0.30000000000000004, 0.6000000000000001, 3)
{67, 12, 36}
(p_rew,p,start) =  (0.30000000000000004, 0.6000000000000001, 4)
{73, 74, 36, 82}
(p_rew,p,start) =  (0.30000000000000004, 0.6000000000000001, 5)
{67, 5, 83, 26, 31}
(p_rew,p,start) =  (0.30000000000000004, 0.6000000000000001, 6)
{36, 5, 39, 9, 47, 17}
(p_rew,p,start) =  (0.30000000000000004, 0.6000000000000001, 7)
{71, 72, 75, 11, 13, 80, 58}
(p_rew,p,start) =  (0.30000000000000004, 0.6000000000000001, 8)
{96, 97, 33, 67, 42, 46, 81, 24}
(p_rew,p,start) =  (0.30000000000000004, 0.6000000000000001, 9)
{35, 37, 11, 44, 13

(p_rew,p,start) =  (0.4, 0.6000000000000001, 1)
{41}
(p_rew,p,start) =  (0.4, 0.6000000000000001, 2)
Initial node infection step failed
None
{2, 50}
(p_rew,p,start) =  (0.4, 0.6000000000000001, 3)
{26, 52, 79}
(p_rew,p,start) =  (0.4, 0.6000000000000001, 4)
Initial node infection step failed
None
{99, 98, 83, 14}
(p_rew,p,start) =  (0.4, 0.6000000000000001, 5)
{0, 65, 98, 35, 72}
(p_rew,p,start) =  (0.4, 0.6000000000000001, 6)
{44, 81, 20, 55, 90, 91}
(p_rew,p,start) =  (0.4, 0.6000000000000001, 7)
{96, 1, 80, 82, 19, 51, 63}
(p_rew,p,start) =  (0.4, 0.6000000000000001, 8)
{65, 34, 33, 4, 38, 84, 22, 63}
(p_rew,p,start) =  (0.4, 0.6000000000000001, 9)
{65, 1, 3, 58, 37, 47, 50, 26, 91}
(p_rew,p,start) =  (0.4, 0.6000000000000001, 10)
{32, 26, 39, 74, 43, 29, 84, 88, 58, 93}
(p_rew,p,start) =  (0.4, 0.7000000000000001, 1)
{67}
(p_rew,p,start) =  (0.4, 0.7000000000000001, 2)
{12, 13}
(p_rew,p,start) =  (0.4, 0.7000000000000001, 3)
Initial node infection step failed
None
Initial node infe

(p_rew,p,start) =  (0.5, 0.6000000000000001, 5)
{68, 41, 11, 14, 93}
(p_rew,p,start) =  (0.5, 0.6000000000000001, 6)
{69, 38, 6, 74, 14, 20}
(p_rew,p,start) =  (0.5, 0.6000000000000001, 7)
{69, 71, 74, 11, 82, 18, 95}
(p_rew,p,start) =  (0.5, 0.6000000000000001, 8)
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
{65, 98, 4, 39, 10, 14, 21, 60}
(p_rew,p,start) =  (0.5, 0.6000000000000001, 9)
Initial node infection step failed
None
Initial node infection step failed
None
{96, 0, 33, 97, 69, 12, 51, 90, 31}
(p_rew,p,start) =  (0.5, 0.6000000000000001, 10)
{96, 3, 67, 99, 37, 17, 82, 52, 20, 26}
(p_rew,p,start) =  (0.5, 0.7000000000000001, 1)
{67}
(p_rew,p,start) =  (0.5, 0.7000000000000001, 2)
{37, 53}
(p_rew,p,start) =  (0.5, 0.7000000000000001, 3)
{4, 44, 87}
(p_rew,p,start) =  (0.5, 0.7000000000000001, 4)
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step fai

(p_rew,p,start) =  (0.6000000000000001, 0.5, 1)
{3}
(p_rew,p,start) =  (0.6000000000000001, 0.5, 2)
{16, 7}
(p_rew,p,start) =  (0.6000000000000001, 0.5, 3)
Initial node infection step failed
None
{27, 83, 6}
(p_rew,p,start) =  (0.6000000000000001, 0.5, 4)
{68, 29, 54, 7}
(p_rew,p,start) =  (0.6000000000000001, 0.5, 5)
{66, 30, 89, 29, 25}
(p_rew,p,start) =  (0.6000000000000001, 0.5, 6)
{43, 82, 58, 91, 93, 31}
(p_rew,p,start) =  (0.6000000000000001, 0.5, 7)
{99, 67, 70, 71, 9, 11, 52}
(p_rew,p,start) =  (0.6000000000000001, 0.5, 8)
{6, 71, 10, 79, 49, 56, 25, 94}
(p_rew,p,start) =  (0.6000000000000001, 0.5, 9)
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
{33, 2, 36, 11, 45, 84, 54, 25, 57}
(p_rew,p,start) =  (0.6000000000000001, 0.5, 10)
Initial node infection step failed
None
{32, 4, 36, 37, 7

(p_rew,p,start) =  (0.7000000000000001, 0.2, 7)
{34, 80, 18, 51, 88, 92, 61}
(p_rew,p,start) =  (0.7000000000000001, 0.2, 8)
{0, 33, 68, 70, 12, 53, 85, 21}
(p_rew,p,start) =  (0.7000000000000001, 0.2, 9)
{35, 37, 70, 72, 8, 76, 79, 21, 27}
(p_rew,p,start) =  (0.7000000000000001, 0.2, 10)
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
{64, 34, 67, 70, 71, 17, 50, 83, 85, 90}
(p_rew,p,start) =  (0.7000000000000001, 0.30000000000000004, 1)
{98}
(p_rew,p,start) =  (0.7000000000000001, 0.30000000000000004, 2)
{85, 70}
(p_rew,p,start) =  (0.7000000000000001, 0.30000000000000004, 3)
{3, 44, 54}
(p_rew,p,start) =  (0.7000000000000001, 0.30000000000000004, 4)
{65, 77, 37, 15}
(p_rew,p,start) =  (0.7000000000000001, 0.30000000000000004, 5)
{33, 68, 43, 20, 24}
(p_rew,p,start) =  (0.7000000000000001, 0.30000000000000004, 6)
{36, 37, 79, 24, 88, 94}
(p_rew,p,start) =  (0.70000000000000

(p_rew,p,start) =  (0.7000000000000001, 1.0, 4)
Initial node infection step failed
None
Initial node infection step failed
None
{32, 20, 76, 95}
(p_rew,p,start) =  (0.7000000000000001, 1.0, 5)
{65, 9, 12, 48, 61}
(p_rew,p,start) =  (0.7000000000000001, 1.0, 6)
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
{98, 86, 55, 26, 92, 95}
(p_rew,p,start) =  (0.7000000000000001, 1.0, 7)
{37, 42, 76, 79, 50, 88, 62}
(p_rew,p,start) =  (0.7000000000000001, 1.0, 8)
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
{34, 9, 10, 46, 15, 81, 19, 31}
(p_rew,p,start) =  (0.7000000000000001, 1.0, 9)
{32, 35, 41, 15, 18, 83, 54, 24, 62}
(p_rew,p,start) =  (0.7000000000000001, 1.0, 10)
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection 

(p_rew,p,start) =  (0.8, 0.6000000000000001, 10)
Initial node infection step failed
None
{34, 3, 71, 45, 78, 82, 21, 22, 59, 29}
(p_rew,p,start) =  (0.8, 0.7000000000000001, 1)
{26}
(p_rew,p,start) =  (0.8, 0.7000000000000001, 2)
{53, 62}
(p_rew,p,start) =  (0.8, 0.7000000000000001, 3)
{72, 5, 55}
(p_rew,p,start) =  (0.8, 0.7000000000000001, 4)
Initial node infection step failed
None
Initial node infection step failed
None
{86, 66, 77, 78}
(p_rew,p,start) =  (0.8, 0.7000000000000001, 5)
Initial node infection step failed
None
Initial node infection step failed
None
{64, 7, 11, 54, 94}
(p_rew,p,start) =  (0.8, 0.7000000000000001, 6)
Initial node infection step failed
None
{66, 3, 73, 52, 54, 29}
(p_rew,p,start) =  (0.8, 0.7000000000000001, 7)
{96, 36, 76, 14, 49, 21, 22}
(p_rew,p,start) =  (0.8, 0.7000000000000001, 8)
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
{35, 69, 45, 16, 19, 83, 54, 29}
(p_rew,p,start) = 

In [ ]:
# '''time series plot of a fixed start but varying p'''
# a = 4 #this is the value of p-1
# for i in np.arange(0,110,10) : 
#     print(a+i)
#     timeseries_infec_frac.loc[a+i,:].plot()
# plt.ylim(-0.1,1.1)
# # plt.xlim(0,10)
# plt.legend(loc=(1,1))
# plt.show()

In [ ]:
# plt.clf()
# '''time series plot of a fixed p but varying start'''
# a = 40 # this is the value of p
# for i in range(0,11) :
#     timeseries_infec_frac.loc[a+i,:].plot()
# plt.ylim(-0.1,1.1)
# # plt.ylim(-0.01,0.11)
# # plt.xlim(0,2.5)
# plt.legend(loc=(1,1))
# plt.show()

In [ ]:
# start_len = 10
# p_len = 11
# X,Y = np.meshgrid([i*0.01 for i in range(11,21)],[i*0.1 for i in range(0,11)])
# print('X = ', X)
# print('Y = ', Y)
# phi = 999*np.ones((p_len,start_len))
# counter = 0
# last_col = np.int64((T/dt)-1)
# for i in range(p_len) :
#     for j in range(start_len) :
#         phi[i,j] = timeseries_infec_frac.loc[counter,(last_col-100)+1:last_col].mean()
#         counter = counter + 1
# # print(counter)
# print(phi)
# fig = plt.gcf()
# fig.set_size_inches(10, 9)
# plt.scatter(X, Y,s=500,c=phi,vmin=0,vmax=1)
# plt.colorbar()
# plt.xlabel(r'$\phi_{ini}$', fontsize = 20)
# plt.ylabel(r'$p\times dt$', fontsize = 20)
# plt.xlim(0.10,0.21)
# plt.title(r'$\phi^*,p_{rew}=0.5,av_{deg}=9,d=1,d^*=3,\gamma=5,T=N=100,dt=0.1,r=1$',fontsize = 15)
# plt.savefig('static_avdeg_9_ini_0_11.svg',format='svg')
# plt.show()

In [ ]:
# filename = 'file_avdeg_'+str(average_degree)+'_ini_'+str(np.int64(X[0,0]*N))+'_'+str(np.int64(X[p_len-1,start_len-1]*N))+'.h5'
# key 
# timeseries_infec_frac.to_hdf(filename, key='timeseries_infec_frac', mode='w')

In [ ]:
# import h5py
# f = h5py.File('file_avdeg_6_ini_0_11.h5', 'r')
# [key for key in f.keys()]
# pd.read_hdf('file_avdeg_6_ini_0_11_firstattemp.h5','p_rew_2')